In [4]:
import json
import os
import time
import pandas as pd
import random
import time

import numpy as np
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding


from keras.backend import clear_session
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
K.set_session(session)

In [5]:
m1 = time.time()
with open("bb_train<20.json") as f:
    data=json.load(f)
print('num samples: ',len(data))
print('json load time: ',time.time()-m1)

#cant be too large, will trigger a memory error later on. Failed at 1 Mill
np.random.shuffle(data)
data = data[:1000000]

print('total time: ',time.time()-m1)

num samples:  55873682
json load time:  4.183709144592285
total time:  26.842750549316406


Need to optimize this later, use np arrays instead of python lists


In [6]:
"""
Need to optimize this later, replace use np arrays 
instead of python lists
"""
top_k = 10000

t = Tokenizer(num_words=top_k, oov_token='<unk>')
t.fit_on_texts(data)



start = time.time()
seqs = [t.texts_to_sequences(x) for x in data]
print('elaspted text->int seqs: ',time.time() - start)

seq_data = list()
for seq in seqs:
    flat = [num for sublist in seq for num in sublist]
    seq_data.append(flat)
    
print('total time: ',time.time() - start)

elaspted text->int seqs:  39.57601809501648
total time:  41.54301404953003


In [10]:
t

For reference, seqs took 57s to go through 1 mill samples

In [7]:
max_len = max([len(seq) for seq in seq_data])

sequences = pad_sequences(seq_data, maxlen=max_len, padding='pre')

print('Max Seq Len: %d' % max_len)

sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
vocab_sz = len(t.word_index)+1
print('vocab sz: ',vocab_sz)

#y_cat = to_categorical(y, num_classes=vocab_sz) #1-hot encoding 

Max Seq Len: 19
vocab sz:  40497


In [8]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.20, random_state=7)

#NN definition
model = Sequential()
model.add(Embedding(vocab_sz, 300, input_length=max_len-1))
model.add(LSTM(50))
model.add(Dense(vocab_sz, activation='softmax'))
print(model.summary())


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 18, 300)           12149100  
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                70200     
_________________________________________________________________
dense_1 (Dense)              (None, 40497)             2065347   
Total params: 14,284,647
Trainable params: 14,284,647
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

#save model structure as json

#model_json = model.to_json()
#with open("models/huffpo-model-v1.json", "w") as json_file:
#    json_file.write(model_json)

In [11]:

model_json = model.to_json()
with open("models/bb-model-v1.json", "w") as json_file:
    json_file.write(model_json)

fpath = "weights/bb-v1/best-weights.hdf5"
checkpoint = ModelCheckpoint(fpath, monitor='val_loss', verbose=2, save_best_only=True, mode='min')


BS = 42
tb = TensorBoard(log_dir="tensorboard-logs/{}".format(time.time()))
callback_lst = [checkpoint, tb]
#steps_per_epoch is num of batches that make up 1 epoch, defaults to size of train set
model.fit(X,y,batch_size=BS, validation_split=.15, epochs=20, callbacks=callback_lst, verbose=1)



Train on 850000 samples, validate on 150000 samples
Epoch 1/20
850000/850000 [==============================] - 701s 825us/step - loss: 0.8093 - acc: 0.7548 - val_loss: 0.7712 - val_acc: 0.7625

Epoch 00001: val_loss improved from inf to 0.77118, saving model to weights/bb-v1/best-weights.hdf5
Epoch 2/20
850000/850000 [==============================] - 700s 823us/step - loss: 0.7269 - acc: 0.7782 - val_loss: 0.7078 - val_acc: 0.7857

Epoch 00002: val_loss improved from 0.77118 to 0.70785, saving model to weights/bb-v1/best-weights.hdf5
Epoch 3/20
850000/850000 [==============================] - 693s 816us/step - loss: 0.6867 - acc: 0.7887 - val_loss: 0.6755 - val_acc: 0.7949

Epoch 00003: val_loss improved from 0.70785 to 0.67555, saving model to weights/bb-v1/best-weights.hdf5
Epoch 4/20
850000/850000 [==============================] - 693s 816us/step - loss: 0.6613 - acc: 0.7962 - val_loss: 0.6510 - val_acc: 0.8011

Epoch 00004: val_loss improved from 0.67555 to 0.65102, saving model

In [13]:
import pickle
with open('bb-v1-tok.pickle', 'wb') as handle:
    pickle.dump(t, handle, protocol=pickle.HIGHEST_PROTOCOL)
